# Imports

In [61]:
import pandas as pd
import os
from glob import glob
from typing import List

# Files

In [62]:
src_path = os.path.join("/", "Volumes", "owen", "datas", "rplace", "processed")
src_glob = os.path.join(src_path, "2022_place_canvas_history-*.csv")
dest_path = os.path.join(src_path, "..")

In [63]:
files = glob(src_glob)
files

['/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000000.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000001.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000002.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000003.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000004.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000005.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000006.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000007.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000008.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000009.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000010.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-00000000001

# Data processing

In [64]:
def count_hours(filename: str):
    df_place = pd.read_csv(filename, index_col=0)
    df_place["date"] = pd.to_datetime(df_place["date"])
    df_place = df_place.resample("1h", on="date").count()
    df_place["hour"] = df_place.index.hour
    df_place.set_index("hour", inplace=True)

    return df_place[["user_id"]].rename({"user_id": "count"}, axis=1).sort_index()

In [65]:
count_hours(filename=files[0])

/Users/owen/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,count
hour,
0,140799
1,1034518


In [66]:
def merge_hours_count(files: List[str]):
    df: pd.DataFrame = count_hours(files[0])

    for i in range(1, len(files)):
        print(f"Adding {files[i]} ({i / len(files) * 100}%)")
        df_to_add = count_hours(files[i])
        df = df.add(df_to_add, fill_value=1)
    
    return df.sort_index()

In [67]:
df = merge_hours_count(files[:])
df

/Users/owen/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000001.csv (1.282051282051282%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000002.csv (2.564102564102564%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000003.csv (3.8461538461538463%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000004.csv (5.128205128205128%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000005.csv (6.41025641025641%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000006.csv (7.6923076923076925%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000007.csv (8.974358974358974%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000008.csv (10.256410256410255%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000009.csv (11.538461538461538%)
Adding /Volumes/owen/data

,count
hour,
0,4960343.0
1,4807651.0
2,4815391.0
3,4582766.0
4,4344703.0
5,3855757.0
6,3293435.0
7,3181065.0
8,3424610.0


In [68]:
df.to_csv(os.path.join(dest_path, "hours_count.csv"))